<a href="https://colab.research.google.com/github/IsraelAbebe/Personal-Projects-and-Exercises/blob/master/AIMS-Assignments/Deep-Learning/Exercise_3_Jan_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!unzip drive/My\ Drive/data/data.zip 

Archive:  drive/My Drive/data/data.zip
   creating: data/
   creating: data/mnist/
   creating: data/mnist/raw/
  inflating: data/mnist/raw/train-images-idx3-ubyte  
  inflating: data/mnist/raw/train-labels-idx1-ubyte  
  inflating: data/mnist/raw/t10k-images-idx3-ubyte  
  inflating: data/mnist/raw/t10k-labels-idx1-ubyte  
   creating: data/mnist/processed/
  inflating: data/mnist/processed/training.pt  
  inflating: data/mnist/processed/test.pt  
   creating: data/mnist/MNIST/
   creating: data/mnist/MNIST/raw/
  inflating: data/mnist/MNIST/raw/train-images-idx3-ubyte  
  inflating: data/mnist/MNIST/raw/train-labels-idx1-ubyte  
  inflating: data/mnist/MNIST/raw/t10k-images-idx3-ubyte  
  inflating: data/mnist/MNIST/raw/t10k-labels-idx1-ubyte  
   creating: data/mnist/MNIST/processed/
  inflating: data/mnist/MNIST/processed/training.pt  
  inflating: data/mnist/MNIST/processed/test.pt  
   creating: data/cifar10/
  inflating: data/cifar10/cifar-10-python.tar.gz  
   creating: data/ci

In [7]:
import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

train_input, train_target, test_input, test_target = \
    prologue.load_data(one_hot_labels = True, normalize = True, flatten = False)

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x



# Train Model

In [0]:
def train(model,train_input,train_target,mini_batch_size,epoch=25,print_line=False):
    criterion = nn.MSELoss()
    eta = 1e-1
    model.train()
    for e in range(0, epoch):
        sum_loss = 0
        # We do this with mini-batches
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            sum_loss = sum_loss + loss.item()
            model.zero_grad()
            loss.backward()
            for p in model.parameters():
                p.data.sub_(eta * p.grad.data)
        if print_line:
            print(e, sum_loss)

In [0]:
model = Net()
train_input, train_target = Variable(train_input), Variable(train_target)
test_input, test_target = Variable(test_input), Variable(test_target)

mini_batch_size = 100

# Error Count

In [0]:
def compute_nb_errors(model,input,target,mini_batch_size):
    model.eval()
    output = model(input)
    correct_count = torch.sum(output.max(1)[1]==target.max(1)[1])
    return 1-(float(correct_count.item())/target.size(0))


In [115]:
for i in range(10):
    model = Net()
    train(model=model,train_input=train_input,train_target=train_target,mini_batch_size=100,epoch=25)
    error = compute_nb_errors(model=model,input=test_input,target=test_target,mini_batch_size=100)
    print('Training ',i,' ,Error:',error)

Training  0  ,Error: 0.18300000000000005
Training  1  ,Error: 0.14900000000000002
Training  2  ,Error: 0.16100000000000003
Training  3  ,Error: 0.15400000000000003
Training  4  ,Error: 0.14500000000000002
Training  5  ,Error: 0.15200000000000002
Training  6  ,Error: 0.15200000000000002
Training  7  ,Error: 0.14400000000000002
Training  8  ,Error: 0.14600000000000002
Training  9  ,Error: 0.16100000000000003


# Modefied CNN

In [0]:
class Net_modefied(nn.Module):
    def __init__(self,n_hidden):
        super(Net_modefied, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(256, n_hidden)
        self.fc2 = nn.Linear(n_hidden, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x

# Influence the number of hiddne Layers

In [117]:
hidden_unit = [10,50,200,500,1000]
for i in range(len(hidden_unit)):
    model = Net_modefied(n_hidden=hidden_unit[i])
    train(model=model,train_input=train_input,train_target=train_target,mini_batch_size=100,epoch=25)
    error = compute_nb_errors(model=model,input=test_input,target=test_target,mini_batch_size=100)
    print('Hidden Units:',hidden_unit[i],'Training ',i,' ,Error:',error)

Hidden Units: 10 Training  0  ,Error: 0.30400000000000005
Hidden Units: 50 Training  1  ,Error: 0.134
Hidden Units: 200 Training  2  ,Error: 0.17500000000000004
Hidden Units: 500 Training  3  ,Error: 0.135
Hidden Units: 1000 Training  4  ,Error: 0.121


# Three Convolutional Layers

In [0]:
class Net_modefied2(nn.Module):
    def __init__(self,n_hidden):
        super(Net_modefied2, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=5)
        self.fc1 = nn.Linear(2*2*64, n_hidden)
        self.fc2 = nn.Linear(n_hidden, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(self.conv2(x))
        x = F.relu(F.max_pool2d(self.conv3(x), kernel_size=2))
        x = F.relu(self.fc1(x.view(-1, 2*2*64)))
        x = self.fc2(x)
        return x

In [151]:
def get_shape(h,p=0,k=5,s=1):
    return ((h+(2*p)-k)/s)+1

get_shape(8)//2

2.0

In [152]:
model = Net_modefied2(n_hidden=200)
train(model=model,train_input=train_input,train_target=train_target,mini_batch_size=100,epoch=25,print_line=True)
error = compute_nb_errors(model=model,input=test_input,target=test_target,mini_batch_size=100)
print('Error:',error)

0 0.965524435043335
1 0.8611425310373306
2 0.8158542960882187
3 0.7699950486421585
4 0.7204816564917564
5 0.6710934266448021
6 0.626787256449461
7 0.5926543325185776
8 0.5653242617845535
9 0.5363439619541168
10 0.5086380988359451
11 0.4865279234945774
12 0.474288672208786
13 0.4537847004830837
14 0.45115071535110474
15 0.41334347054362297
16 0.40101541578769684
17 0.3879912942647934
18 0.37704773992300034
19 0.3691019006073475
20 0.35120957531034946
21 0.3512778840959072
22 0.33306819200515747
23 0.3166050259023905
24 0.31706047989428043
Error: 0.15800000000000003
